# Fine-Tuning GPT for Yandex Reviews

Данный проект демонстрирует процесс дообучения (Fine-Tuning) предобученной русскоязычной GPT-модели на датасете отзывов, взятом из репозитория [Yandex Geo Reviews Dataset 2023](https://www.kaggle.com/datasets/kyakovlev/yandex-geo-reviews-dataset-2023).

## Содержимое

1. **`FineTune_Yandex.ipynb`** - Jupyter Notebook, в котором осуществляется:
   - Загрузка датасета с Kaggle (через `kaggle.json`).
   - Предобработка и очистка данных.
   - Балансировка классов (пример).
   - Дообучение (fine-tuning) GPT-модели.
   - Сохранение дообученных весов.
   - Примеры генерации отзывов.

2. **`trained_model/`** - Папка (или архив), содержащая дообученные веса модели (Checkpoints), полученные в результате обучения.  
   - Может включать файлы `pytorch_model.bin`, `config.json`, `tokenizer_config.json`, `vocab.json`, `merges.txt` и т.д.

3. **`requirements.txt`** - список необходимых библиотек.

## Запуск

1. Создайте окружение (например, conda или виртуальное окружение Python) и установите библиотеки из `requirements.txt`.
2. Запустите ноутбук `FineTune_Yandex.ipynb` (например, в Google Colab или локально).
3. При желании, измените параметры обучения (число эпох, `batch_size`, `max_length` и т.д.) в соответствующих ячейках.

## Генерация отзывов

После обучения вы можете вызвать функцию `generate_review(rating, rubrics, ...)`, чтобы сгенерировать отзывы на русском языке. Пример вызова в ноутбуке:

```python
generate_review(5.0, "Кафе;Ресторан", num_return_sequences=3)
```

## Установка и импорт библиотек

In [1]:
# Системные
import os
import sys
import random

# Работа с данными
import numpy as np
import pandas as pd

# Для визуализации
import matplotlib.pyplot as plt

# HuggingFace Transformers
!pip install --upgrade pip
!pip install numpy pandas scikit-learn matplotlib
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 # или cuNN, в зависимости от CUDA
!pip install transformers datasets accelerate
!pip install kaggle

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling
from transformers import pipeline

import torch
import re
from tqdm.auto import tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207

## Загрузка датасета с Kaggle

 - Для работы с kaggle необходимо иметь файл `kaggle.json` с API токеном.
 - В Colab нужно загрузить этот файл через интерфейс или напрямую из локальной машины.

Данный код копирует файл kaggle.json в нужное место и скачивает датасет.
Если вы запускаете локально и уже имеете датасет, пропустите этот блок и
укажите путь к датасету вручную.


In [2]:
from google.colab import files

# Загрузим kaggle.json (API токен) в среду Colab:
uploaded = files.upload()  # выберите файл kaggle.json

# Создадим папку .kaggle и переместим токен
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# Убедимся, что kaggle установлен
! kaggle --version

# Скачаем датасет
DATASET_KAGGLE_NAME = "kyakovlev/yandex-geo-reviews-dataset-2023"
! kaggle datasets download -d {DATASET_KAGGLE_NAME}

# Распакуем архив
! unzip -o yandex-geo-reviews-dataset-2023.zip

# После распаковки нужно проверить, как называется файл (csv или tskv)

Saving kaggle.json to kaggle.json
Kaggle API 1.6.17
Dataset URL: https://www.kaggle.com/datasets/kyakovlev/yandex-geo-reviews-dataset-2023
License(s): other
 96% 191M/200M [00:02<00:00, 119MB/s]
100% 200M/200M [00:02<00:00, 96.4MB/s]
Archive:  yandex-geo-reviews-dataset-2023.zip
  inflating: AUTHORS.md              
  inflating: LICENSE.md              
  inflating: README.md               
  inflating: geo-reviews-dataset-2023.csv  
  inflating: geo-reviews-dataset-2023.tskv  


## Подготовка и первичная предобработка

В результате распаковки у нас есть файлы
'geo-reviews-dataset-2023.csv' и 'geo-reviews-dataset-2023.tskv'.

In [3]:
dataset_path_csv = "geo-reviews-dataset-2023.csv"
dataset_path_tskv = "geo-reviews-dataset-2023.tskv"

if os.path.exists(dataset_path_tskv):
    DATA_FILE = dataset_path_tskv
elif os.path.exists(dataset_path_csv):
    DATA_FILE = dataset_path_csv
else:
    # Если мы здесь — возможно, нужный файл так не называется.
    # Можно вывести список файлов в текущей папке и разобраться.
    print("Файлы в текущей директории:", os.listdir("."))
    raise FileNotFoundError("Не найден файл geo-reviews-dataset-2023.csv или .tskv. Проверьте название!")

print(f"Используем датасет: {DATA_FILE}")

# Попытаемся прочитать датасет. В оригинале это tskv-файл (таб-разделённый), где ключ=значение.
# Но есть и CSV-версия. Проверим формат.
# Для TSKV:
def read_tskv_file(path):
    data = []
    with open(path, 'r', encoding='utf-8') as file:
        for line in file:
            pairs = line.strip().split('\t')
            record = {}
            for pair in pairs:
                if '=' in pair:
                    key, value = pair.split('=', 1)
                    record[key.strip()] = value.strip()
            if len(record) > 0:
                data.append(record)
    return pd.DataFrame(data)

df = None
if DATA_FILE.endswith(".tskv"):
    df = read_tskv_file(DATA_FILE)
else:
    # Попробуем читать CSV (разделитель может быть разный, но чаще всего ',')
    df = pd.read_csv(DATA_FILE, sep=',', encoding='utf-8')

print("Первые 5 строк:")
print(df.head())

print("Колонки:", df.columns)

Используем датасет: geo-reviews-dataset-2023.tskv
Первые 5 строк:
                                             address             name_ru  \
0  Екатеринбург, ул. Московская / ул. Волгоградск...  Московский квартал   
1  Московская область, Электросталь, проспект Лен...   Продукты Ермолино   
2  Краснодар, Прикубанский внутригородской округ,...             LimeFit   
3   Санкт-Петербург, проспект Энгельса, 111, корп. 1        Snow-Express   
4                  Тверь, Волоколамский проспект, 39  Студия Beauty Brow   

  rating                                            rubrics  \
0     3.                                     Жилой комплекс   
1     5.  Магазин продуктов;Продукты глубокой заморозки;...   
2     1.                                        Фитнес-клуб   
3     4.        Пункт проката;Прокат велосипедов;Сапсёрфинг   
4     5.  Салон красоты;Визажисты, стилисты;Салон бровей...   

                                                text  
0  Московский квартал 2.\nШумно : летом по 

## Очистка и дополнительная предобработка

In [4]:
# Проверим, какие колонки нам нужны. Мы хотим rating, rubrics, text.
# Убедимся, что они есть в датасете.
required_cols = ["rating", "rubrics", "text"]
for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"Не найдена колонка '{col}' в датасете. Доступные колонки: {df.columns}")

# Удалим строки, где text или rating пусты/отсутствуют
df.dropna(subset=["text", "rating"], inplace=True)

# Преобразуем rating к float (если не float)
df["rating"] = df["rating"].astype(str)
df["rating"] = df["rating"].str.replace(",", ".", regex=False)
df["rating"] = df["rating"].str.replace("[^0-9\\.]", "", regex=True)  # оставим только цифры и точки
df["rating"] = pd.to_numeric(df["rating"], errors='coerce')
df.dropna(subset=["rating"], inplace=True)

# Удалим экстремально короткие/длинные тексты
df["text"] = df["text"].astype(str)
df = df[df["text"].str.len() >= 10]
df = df[df["text"].str.len() <= 1500]

# Для удобства создадим признак класса (бинарный или тернарный) на основе рейтинга,
# чтобы можно было сделать балансировку. Например:
#  - rating >= 4.0: "positive"
#  - 2.5 <= rating < 4.0: "neutral"
#  - rating < 2.5: "negative"
# (Это лишь пример: вы можете сделать и более тонкую градацию.)
def get_rating_class(r):
    if r >= 4.0:
        return "positive"
    elif r >= 2.5:
        return "neutral"
    else:
        return "negative"

df["rating_class"] = df["rating"].apply(get_rating_class)

# Посмотрим на распределение классов
print("Распределение rating_class:")
print(df["rating_class"].value_counts())

Распределение rating_class:
rating_class
positive    429617
negative     44741
neutral      21138
Name: count, dtype: int64


## Балансировка классов

In [5]:
"""
У нас есть классический дисбаланс (положительных отзывов больше, чем нейтральных и отрицательных).
Для наглядности применим простейший вариант балансировки — undersampling (выравниваем все классы по минимальному количеству).
"""

class_counts = df["rating_class"].value_counts()
min_count = class_counts.min()

balanced_df = pd.DataFrame()
for cls in class_counts.index:
    subset = df[df["rating_class"] == cls]
    # Выберем из каждого класса не больше min_count
    subset_sampled = subset.sample(n=min_count, replace=False, random_state=42)
    balanced_df = pd.concat([balanced_df, subset_sampled], axis=0)

balanced_df = balanced_df.sample(frac=1.0, random_state=42).reset_index(drop=True)
print("После балансировки:")
print(balanced_df["rating_class"].value_counts())

df = balanced_df  # далее работаем с сбалансированным датасетом

После балансировки:
rating_class
negative    21138
neutral     21138
positive    21138
Name: count, dtype: int64


## Формирование промптов (prompt) для генерации

In [6]:
"""
Мы хотим, чтобы модель "знала", какой рейтинг и рубрики у отзыва, и на их основе
генерировала текст отзыва. Для этого мы дополним начало строки промптом вида:
    "Rating: 4.5 | Rubrics: Кафе;Ресторан\nОтзыв: "
Затем в обучение будем подавать "prompt + text", чтобы модель училась продолжать.

В дальнейшем, при генерации, мы будем подавать "prompt" и просить модель генерировать продолжение.
"""

def create_prompt(rating, rubrics):
    return f"Rating: {rating:.1f} | Rubrics: {rubrics}\nОтзыв: "

df["prompt"] = df.apply(lambda row: create_prompt(row["rating"], row["rubrics"]), axis=1)
df["full_text"] = df["prompt"] + df["text"]

# Посмотрим пример
print("\nПример поля full_text:")
print(df["full_text"].iloc[0][:300], "...")


Пример поля full_text:
Rating: 2.0 | Rubrics: Быстрое питание;Столовая
Отзыв: Неплохое предприятие общественного питания, особенно пользуется спросом у студентов из- за быстрого приготовления современных бурго бутербродов и т.д., но на территорию, чуть ли не под навес, заезжает столько машин, приезжающих перекусить сюда и ...


## Создание HuggingFace Dataset и токенизация

In [7]:
# Оставим только нужное для обучения поле "full_text"
df_for_hf = df[["full_text"]].copy()
hf_dataset = Dataset.from_pandas(df_for_hf)

# Выбираем предобученную модель. Для русскоязычных GPT есть несколько вариантов:
# - sberbank-ai/rugpt3small_based_on_gpt2
# - ai-forever/rugpt3small
# - sberbank-ai/rugpt2large
# и т.д.
model_name = "sberbank-ai/rugpt3small_based_on_gpt2"

print(f"Инициализируем токенизатор {model_name}")
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# GPT2 обычно не имеет pad_token. Чтобы избежать ошибок, присвоим его = eos_token
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(batch):
    return tokenizer(
        batch["full_text"],
        truncation=True,
        max_length=512,  # можно варьировать
        padding="max_length"
    )

print("Запуск токенизации...")
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True, remove_columns=["full_text"])

# Разделим на train/test, например 90/10
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

print(f"Train size: {len(train_dataset)}, Eval size: {len(eval_dataset)}")

Инициализируем токенизатор sberbank-ai/rugpt3small_based_on_gpt2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

Запуск токенизации...


Map:   0%|          | 0/63414 [00:00<?, ? examples/s]

Train size: 57072, Eval size: 6342


## Инициализация модели

In [8]:
print(f"Загружаем модель {model_name}")
model = GPT2LMHeadModel.from_pretrained(model_name)

Загружаем модель sberbank-ai/rugpt3small_based_on_gpt2


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

## Подготовка Trainer и DataCollator

In [9]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./trained_model",
    overwrite_output_dir=True,
    evaluation_strategy="steps",
    eval_steps=200,              # каждые N шагов делаем оценку
    logging_steps=100,           # каждые N шагов логируем
    save_steps=500,              # как часто сохранять чекпоинты
    num_train_epochs=1,          # число эпох (для демонстрации 1; для реальной задачи увеличьте)
    per_device_train_batch_size=2,  # настройте под вашу GPU
    per_device_eval_batch_size=2,
    warmup_steps=100,            # "разогрев" lr
    weight_decay=0.01,           # L2 regularization
    learning_rate=5e-5,
    fp16=True if torch.cuda.is_available() else False,  # использовать 16-бит, если GPU позволяет
    logging_dir="./logs",
    report_to="none",            # или "tensorboard"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Обучение

In [10]:
print("Начинаем обучение...")
trainer.train()

Начинаем обучение...


Step,Training Loss,Validation Loss
200,3.253800,3.162656
400,3.229100,3.117531
600,3.168500,3.099224
800,3.160300,3.089185
1000,3.247800,3.077691
1200,3.083300,3.071135
1400,3.247900,3.058250
1600,3.137100,3.052169
1800,3.079200,3.060187
2000,3.173200,3.038123


KeyboardInterrupt: 

## Сохранение итоговой модели

In [11]:
print("Сохраняем дообученную модель в папку './trained_model'...")
trainer.save_model("./trained_model")

Сохраняем дообученную модель в папку './trained_model'...


## Пример генерации

In [14]:
"""
Покажем, как использовать дообученную модель для генерации отзывов.

Мы будем подавать строку-промпт вида:
    "Rating: 5.0 | Rubrics: Ресторан\nОтзыв: "
и просить модель сгенерировать продолжение.
"""

print("\n=== Пример генерации отзывов ===")
import re
import pandas as pd
import torch
from IPython.display import display, HTML
from transformers import pipeline

# Если модель не загружена, создаём пайплайн:
device = 0 if torch.cuda.is_available() else -1
gen_pipeline = pipeline(
    "text-generation",
    model="./trained_model",  # путь к обученной модели
    tokenizer=tokenizer,      # ваш GPT2Tokenizer
    device=device
)

########################################
# Функция для определения тональности
# (опционально, если нужно это использовать)
########################################

def get_sentiment_label(rating):
    """
    Определяет тональность отзыва на основе рейтинга.
    Возвращает строку 'positive', 'neutral' или 'negative'.
    """
    if rating >= 4.0:
        return "positive"
    elif rating >= 2.5:
        return "neutral"
    else:
        return "negative"

########################################
# Функция очистки сгенерированного текста
########################################

def clean_generated_text(text):
    """
    Удаляет лишние пробелы, спецсимволы переноса и обрезает на последнем знаке
    пунктуации (. ! ?). Можно доработать по вкусу.
    """
    # Заменяем все пробельные символы на пробел
    text = re.sub(r'\s+', ' ', text)

    # Удаляем повторные переносы строк, если они каким-то образом остались
    text = text.replace('\\n', ' ').replace('\n', ' ')

    # Находим последний знак препинания (. ? !) и обрезаем до него (вместе с ним)
    match = list(re.finditer(r'[.!?]', text))
    if match:
        last_punc_index = match[-1].end()
        text = text[:last_punc_index]

    return text.strip()

########################################
# Функция генерации отзыва
########################################

def generate_reviews(params_list,
                     max_length=100,
                     num_return_sequences=1,
                     do_sample=True,
                     top_k=50,
                     top_p=0.9,
                     temperature=1.0,
                     repetition_penalty=1.2,
                     no_repeat_ngram_size=2):
    """
    Генерирует отзывы для списка параметров (rating, rubrics).

    params_list: список словарей вида [{"rating": <float>, "rubrics": <str>}, ...]
    Возвращает pd.DataFrame с результатами (rating, rubrics, sentiment, generated_text).
    """
    results = []

    for param in params_list:
        rating = param.get("rating", 3.0)
        rubrics = param.get("rubrics", "Не указано")

        # Опционально добавим &laquo;тональность&raquo;
        sentiment_label = get_sentiment_label(rating)

        # Формируем промпт
        prompt = f"Rating: {rating} | Rubrics: {rubrics}\nОтзыв ({sentiment_label}): "

        # Генерируем продолжение
        outputs = gen_pipeline(
            prompt,
            max_length=max_length,
            num_return_sequences=num_return_sequences,
            do_sample=do_sample,
            top_k=top_k,
            top_p=top_p,
            temperature=temperature,
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=no_repeat_ngram_size
        )

        # Сохраняем результаты в виде списка
        for out in outputs:
            full_text = out["generated_text"]
            # Удаляем из результата сам prompt, чтобы оставить &laquo;чистый&raquo; отзыв
            generated_part = full_text.replace(prompt, "")
            # Чистим итоговый текст
            cleaned = clean_generated_text(generated_part)

            results.append({
                "rating": rating,
                "rubrics": rubrics,
                "sentiment": sentiment_label,
                "generated_text": cleaned
            })

    # Превращаем результаты в DataFrame
    df_results = pd.DataFrame(results)
    return df_results

########################################
# Пример использования
########################################

example_params = [
    {"rating": 5.0, "rubrics": "Кафе;Ресторан"},
    {"rating": 2.0, "rubrics": "Автосервис"},
    {"rating": 3.5, "rubrics": "Кинотеатр"}
]

df_generated = generate_reviews(
    params_list=example_params,
    max_length=120,           # можно увеличить
    num_return_sequences=2,   # по 2 отзыва на каждый вариант
    do_sample=True,
    top_k=50,
    top_p=0.9,
    temperature=0.95,
    repetition_penalty=1.2,
    no_repeat_ngram_size=2
)

# Выводим результат в виде таблицы
print("=== Сгенерированные отзывы ===")
display(HTML(df_generated.to_html(index=False)))


=== Пример генерации отзывов ===


Device set to use cuda:0


=== Сгенерированные отзывы ===


rating,rubrics,sentiment,generated_text
5.0,Кафе;Ресторан,positive,"Построили себе небольшой магазин и начали с мужем готовить в нем пиццу, но тут выяснилось что за эти деньги можно приобрести все то,что мы пробовали. Впервые зашли за 200 рублей(по рекомендации, а потом оказалось,что не вкусно...\l) на улице уже было холодно (потому что у нас был холодина,а так же из-за жары,и тогда по приезду решили сделать ремонт в подвале )."
5.0,Кафе;Ресторан,positive,"Мне очень понравилось, что этот заведение находится рядом с кинотеатром."
2.0,Автосервис,negative,"Сервис хороший, но нет обслуживания. Все что нужно нареканий и претензий к работе этого специалиста Но вот на месте с этим сервисом есть!!!"
2.0,Автосервис,negative,"Записались на диагностику автомобиля, в итоге попали во время ремонта с проблемой на стенде ККМЗ после обращения за капотом, тк к там не было никакой замены масла и гидроусилителя руля, так как не могли установить запчасть (хотя обещали, что он устанавливался в прошлом году)."
3.5,Кинотеатр,neutral,"В этом кинотеатре в основном кино, а не видеопродукция для детей или видеочат из интернета, так вот очень жаль что в нем нет телевизора и к нему нет доступа к интернету."
3.5,Кинотеатр,neutral,"Отличная киношка, хорошая техника, качественные картинки, хорошее качество, звук хороший! Вокзал не плохой, качество плохое. Все остальное на уровне, даже хорошие. Слушали «Территорию парка» с детьми и то хорошо слышно очень громко - по записи, но за это спасибо!"
